In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd

In [14]:
df=pd.read_csv("/content/drive/MyDrive/chatgpt_style_reviews_dataset.xlsx - Sheet1 (1).csv")

In [15]:
df

,date,title,review,rating,username,helpful_votes,review_length,platform,language,location,version,verified_purchase
0,2024-09-06,Claim who accept.,Every quite sense including six lot have never...,4,morrowthomas,30,22,Flipkart,ar,Nepal,3.8.4,No
1,3/7/2025,Growth pretty wish.,Ask develop bag also his worker pass. Expert w...,4,sheakimberly,120,22,Flipkart,ar,Guinea,5.2.6,Yes
2,########,What then spend offer reason whom none.,If customer address region try near risk next ...,5,katherineali,130,21,Flipkart,es,Kuwait,3.6.5,No
3,########,Say dog drug enter director strong student.,To television loss election him small detail r...,1,eric11,72,26,Amazon,pt,Uruguay,5.7.2,No
4,########,Purpose here beyond.,Pass share must amount lot per manage world to...,5,chloe42,123,27,Amazon,zh,Mayotte,5.8.6,No
...,...,...,...,...,...,...,...,...,...,...,...,...
245,########,Father who tax.,Government card history suddenly save theory s...,5,amassey,109,22,App Store,en,Micronesia,2.3.4,Yes
246,########,Remain movement bit step direction give reduce.,Court control million hundred offer total hit ...,5,stevenandrews,26,23,Website,fr,Austria,3.5.3,No
247,########,Form now then watch certain.,Just opportunity ask yet against large practic...,1,michelle90,67,24,App Store,ru,Rwanda,3.9.6,No
248,3/7/2024,Recent catch gas benefit great religious.,Onto simple audience as including claim create...,5,victor24,29,19,Amazon,zh,Iran,4.9.2,No


In [16]:
df.shape

(250, 12)

In [20]:
df.isnull().sum()

,0
date,0
title,0
review,0
rating,0
username,0
helpful_votes,0
review_length,0
platform,0
language,0
location,0


In [25]:
df["title_review"] = df["title"]+ ". " + df["review"]

In [31]:
df.head()

,date,title,review,rating,username,helpful_votes,review_length,platform,language,location,version,verified_purchase,title_review
0,2024-09-06,Claim who accept.,Every quite sense including six lot have never...,4,morrowthomas,30,22,Flipkart,ar,Nepal,3.8.4,No,Claim who accept.. Every quite sense including...
1,3/7/2025,Growth pretty wish.,Ask develop bag also his worker pass. Expert w...,4,sheakimberly,120,22,Flipkart,ar,Guinea,5.2.6,Yes,Growth pretty wish.. Ask develop bag also his ...
2,########,What then spend offer reason whom none.,If customer address region try near risk next ...,5,katherineali,130,21,Flipkart,es,Kuwait,3.6.5,No,What then spend offer reason whom none.. If cu...
3,########,Say dog drug enter director strong student.,To television loss election him small detail r...,1,eric11,72,26,Amazon,pt,Uruguay,5.7.2,No,Say dog drug enter director strong student.. T...
4,########,Purpose here beyond.,Pass share must amount lot per manage world to...,5,chloe42,123,27,Amazon,zh,Mayotte,5.8.6,No,Purpose here beyond.. Pass share must amount l...


In [32]:
df = df.drop_duplicates(subset=['title_review'])

In [33]:
df.head()

,date,title,review,rating,username,helpful_votes,review_length,platform,language,location,version,verified_purchase,title_review
0,2024-09-06,Claim who accept.,Every quite sense including six lot have never...,4,morrowthomas,30,22,Flipkart,ar,Nepal,3.8.4,No,Claim who accept.. Every quite sense including...
1,3/7/2025,Growth pretty wish.,Ask develop bag also his worker pass. Expert w...,4,sheakimberly,120,22,Flipkart,ar,Guinea,5.2.6,Yes,Growth pretty wish.. Ask develop bag also his ...
2,########,What then spend offer reason whom none.,If customer address region try near risk next ...,5,katherineali,130,21,Flipkart,es,Kuwait,3.6.5,No,What then spend offer reason whom none.. If cu...
3,########,Say dog drug enter director strong student.,To television loss election him small detail r...,1,eric11,72,26,Amazon,pt,Uruguay,5.7.2,No,Say dog drug enter director strong student.. T...
4,########,Purpose here beyond.,Pass share must amount lot per manage world to...,5,chloe42,123,27,Amazon,zh,Mayotte,5.8.6,No,Purpose here beyond.. Pass share must amount l...


In [34]:
df.shape

(250, 13)

In [38]:
df['title_review']=df['title_review'].str.lower()
df["title_review"]

,title_review
0,claim who accept.. every quite sense including...
1,growth pretty wish.. ask develop bag also his ...
2,what then spend offer reason whom none.. if cu...
3,say dog drug enter director strong student.. t...
4,purpose here beyond.. pass share must amount l...
...,...
245,father who tax.. government card history sudde...
246,remain movement bit step direction give reduce...
247,form now then watch certain.. just opportunity...
248,recent catch gas benefit great religious.. ont...


In [39]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [40]:
from nltk import word_tokenize

df['title_review']=df['title_review'].apply(lambda X: word_tokenize(X))
df['title_review'].head()

,title_review
0,"[claim, who, accept, .., every, quite, sense, ..."
1,"[growth, pretty, wish, .., ask, develop, bag, ..."
2,"[what, then, spend, offer, reason, whom, none,..."
3,"[say, dog, drug, enter, director, strong, stud..."
4,"[purpose, here, beyond, .., pass, share, must,..."


In [44]:
q`from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [47]:
def lemmatization(title_review):
  result=[][47]
0s

  wordnet = WordNetLemmatizer()
  for token,tag in pos_tag("title_review"):
    # print(token)
    # print(tag)
    pos=tag[0].lower()
    if pos not in ['a', 'r', 'n', 'v']:
      pos='n'
    result.append(wordnet.lemmatize(token,pos))
  return result

In [53]:
#Test
text=df["tittle_review"]
lemmatization(text)

KeyError: 'tittle_review'